# CM50270 Reinforcement Learning
## Coursework Part 1: Value Iteration

In this exercise, you will implement the value iteration algorithm for three closely related, but different, gridworld environments.

**Total number of marks:** 20 marks.

**What to submit:** Your completed Jupyter notebook (.ipynb file) which should include **all** of your source code. Please **do not change the file name or compress/zip your submission**. Please do not include any identifying information on the files you submit. This coursework will be marked **anonymously**.

**Where to submit:** CM50270 Moodle Page.

You are required to **work individually**. You are welcome to discuss ideas with others but you must design your own implementation and **write your own code**.

**Do not plagiarise**. Plagiarism is a serious academic offence. For details on what plagiarism is and how to avoid it, please visit the following webpage: http://www.bath.ac.uk/library/help/infoguides/plagiarism.html

If you are asked to use specific variable names, data-types, function signatures and notebook cells, **please ensure that you follow these instructions**. Not doing so will cause the automarker to reject your work, and will assign you a score of zero for that question. **If the automarker rejects your work because you have not followed the instructions, you may not get any credit for your work**.

Please **do not use any non-standard, third-party libraries** apart from numpy and matplotlib. **If we are unable to run your code because you have used unsupported external libraries, you may not get any credit for your work.**

Please remember to **save your work regularly**.

Please be sure to **restart the kernel and run your code from start-to-finish** (Kernel → Restart & Run All) before submitting your notebook. Otherwise, you may not be aware that you are using variables in memory that you have deleted.

Your total runtime must be less than **1 minute** on the University's computers. Otherwise, you may not get credit for your work. You can run your code on the university's computers remotely using [UniDesk](https://bath.topdesk.net/tas/public/ssp/content/detail/knowledgeitem?unid=ff3266344c1d4eb2acb227cc9e3e1eee).

## Introduction
In this coursework, you will implement the Value Iteration algorithm to compute an optimal policy for three different (but closely related) Markov Decision Processes. For your reference, the pseudo-code for the Value Iteration algorithm is reproduced below from the textbook (Reinforcement Learning, Sutton & Barto, 2018, pp. 83).

<img src="images/value_iteration.png" style="width: 800px;"/>

Please note the following about the pseudo-code: The set $\mathcal{S}$ contains all non-terminal states, whereas $\mathcal{S}^+$ is the set of all states (terminal and non-terminal). The symbol $r$ represents the immediate reward on transition from state $s$ to the next state $s'$ via action $a$. 

<img src="images/bombs and gold numbers.png" style="width: 300px;" align="left" caption="Figure 1"/>

The three problems you will solve use variants of the gridworld environment shown on the left. You should be familiar with this kind of environment from the lectures. The grid squares in the figure are numbered as shown. In all exercises, the following are true: 

**Actions available:** The agent has four possible actions in each grid square. These are *west*, *north*, *south*, and *east*. If the direction of movement is blocked by a wall (for example, if the agent executes action south at grid square 1), the agent remains in the same grid square. 

**Collecting gold:** On its first arrival at a grid square that contains gold (from a neighbouring grid square), the agent collects the gold. Note that, in order to collect the gold, the agent needs to transition into the grid square (containing the gold) from a different grid square.

**Hitting the bomb:** On arrival at a grid square that contains a bomb (from a neighbouring grid square), the agent activates the bomb. 

**Terminal states:** The game terminates when all gold is collected or when the bomb is activated. In Exercises 1 and 2, you can define terminal states to be grid squares 18 and 23. In Exercise 3, you will need to define terminal state(s) differently.


### Instructions ###
Set parameter $\theta$ to $1 \times 10 ^{-10}$. You can express that as `1e-10` in Python. 

Set all initial state values $V(s)$ to zero.

Do not use discounting (that is, set $\gamma=1$).

Use the following reward function: $-1$ for each navigation action (including when the action results in hitting the wall), an additional $+10$ for collecting each piece of gold, and an additional $-10$ for activating the bomb. For example, the immediate reward for transitioning into a square with gold (from a neighbouring grid square) is $-1 + 10 = +9$. 

## Exercise 1: Deterministic Environment (0 Marks)

In this exercise, the agent is able to move in the intended direction with certainty. For example, if it executes action _north_ in grid square 0, it will transition to grid square 5 with probability 1. In other words, we have a deterministic environment.

Compute the optimal policy using Value Iteration. 

Your need to produce two one-dimensional numpy arrays with names `policy` and `v`. Both arrays should have a length of 25, with the element at index $i$ representing grid cell $i$ (see figure above). Both arrays should be accessible in the "solution cell" below!

The array `policy` should be a numpy array of strings that specifies an optimal action at each grid location. Please use the abbreviations `"n"`, `"e"`, `"s"`, and `"w"` for the four actions. As an example, the value of `policy` at index `0` needs to give `"n"`, if _north_ is an optimal action in cell 0. The policy for a terminal state can be any action. If there are multiple optimal actions from a state, any optimal action will be considered as a correct answer. 

The array `v` should be an array of floats that contains the expected return at each grid square (that is, the state value under the optimal policy).

In [1]:
# Please write your code for Exercise 1 in this cell or in as many cells as you want ABOVE this cell.
# Your code should compute the values of policy and v from scratch when this cell is executed, 
# using the value iteration algorithm. We will mark your coursework by checking the values of 
# the variables policy and v in the hidden test cell further below. Do NOT delete this cell.

# YOUR CODE HERE
import numpy as np

class Gridworld_Det():
    def __init__(self,rows=5,coloumns=5,bombpos=[18],goldpos=[23]):
        self.num_r = rows
        self.num_c = coloumns
        self.shape = (self.num_r*self.num_c)
        self.bombpos = bombpos
        self.goldpos = goldpos
        self.num_fields = range(self.num_r * self.num_c)
        self.gold_reward = 10
        self.bomb_reward = -10
        self.terminalstates = self.bombpos + self.goldpos
        self.actionreward = -1
        self.actions = ["n", "e", "s", "w"]
        self.num_actions = len(self.actions)
        self.reward = np.zeros(shape=(self.shape))
        self.val = np.zeros(shape=(self.shape))
        self.policy = np.empty(25, dtype=np.unicode)
        self.gamma = 1
        self.theta = 1e-10
        
    def Reward_propagation(self, starting_pos, action):
        new_position = starting_pos
        reward = 0
        if starting_pos in self.terminalstates:
            return starting_pos,0
        else:
            if action == "n":
                candidate_pos = starting_pos + self.num_c
                new_position = starting_pos if starting_pos + 5 >= self.shape else candidate_pos
                reward = self.actionreward
            elif action == "e":
                candidate_pos = starting_pos + 1 
                new_position = starting_pos if (starting_pos + 1) % 5 == 0 else candidate_pos
                reward = self.actionreward
            elif action == "s":
                candidate_pos = starting_pos - self.num_c
                new_position = starting_pos if starting_pos - 5 < 0 else candidate_pos
                reward = self.actionreward
            elif action == "w":
                candiate_pos = starting_pos - 1
                new_position = starting_pos if (starting_pos) % 5 == 0 else candiate_pos
                reward = self.actionreward
            if new_position == self.goldpos[0]:
                reward += self.gold_reward 
            elif new_position == self.bombpos[0]:
                reward += self.bomb_reward 
        return new_position, reward
        
        
    def ValueIteration(self):
        while True:
            delta = 0
            prev = self.val.copy()
            for state in self.num_fields:
                new_reward=np.zeros(4)
                #for act in range(len(self.actions)):
                for i,act in enumerate(self.actions):
                    new_position,possible_reward = self.Reward_propagation(state,act)
                    new_reward[i] += 1*(possible_reward+self.gamma*self.val[new_position])
                self.val[state] = max(new_reward)
            delta = np.fabs(self.val-prev).max()
           # print(delta)
            if delta < self.theta:
                break
        return self.val
        
    def PolicyIteration(self):
        prev = self.val.copy()
        for state in self.num_fields:
            Policy_reward = []
            #print(Policy_reward.dtype)
            for act in self.actions:
                new_position,possible_reward = self.Reward_propagation(state,act)
                Policy_reward.append(possible_reward+self.gamma*self.val[new_position])
            Policy_reward = np.array(Policy_reward)
            Exp_reward = np.argmax(Policy_reward)
            self.policy[state] = self.actions[Exp_reward]
        return self.policy
   
                
                
            
        
        
environment = Gridworld_Det()



v = environment.ValueIteration()
policy = environment.PolicyIteration()

#policy = print(environment.PolicyIteration().reshape(5,5))
#print(environment.action_reward(10,3))


**Example Test Cell**

In the code cell below, we have provided an example of the type of test code that we will use to mark your work. In these tests, we first check that your `policy` and `v` variables are of the correct type, and then check that their values match the solution. In the future, the test code will be hidden from you.

You must not delete or modify test cells in any way - any modifications you do make will be overwritten at run-time.

In [2]:
# DO NOT DELETE OR MODIFY THIS CELL!
# Your code for Exercise 1 is tested here.

import numpy as np

# We're giving you the solution values for Exercise 1, but not telling you how to compute them!
solution_values = [3.0, 4.0, 5.0, 4.0, 5.0,
                   4.0, 5.0, 6.0, 5.0, 6.0,
                   5.0, 6.0, 7.0, 6.0, 7.0,
                   6.0, 7.0, 8.0, 0.0, 8.0,
                   7.0, 8.0, 9.0, 0.0, 9.0]
solution_values = np.array(solution_values)

# We're giving you the solution policy for Exercise 1, but not telling you how to compute it!
solution_policy = [
                   'n', 'n', 'n', 'n', 'n',
                   'n', 'n', 'n', 'n', 'n',
                   'n', 'n', 'n', 'e', 'n',
                   'n', 'n', 'n', 'n', 'n',
                   'e', 'e', 'e', 'n', 'w',]
solution_policy = np.array(solution_policy)

# Check that policy and v are numpy arrays.
assert(isinstance(policy, np.ndarray))
assert(isinstance(v, np.ndarray))

# Check correct shapes of numpy arrays.
assert(policy.shape == (25, ))
assert(v.shape == (25, ))

# Check whether the numpy arrays have the correct data types.
assert(np.issubdtype(policy.dtype, np.unicode_)) # policy.dtype should be '<U1'
assert(np.issubdtype(v.dtype, np.float64))

# Check whether policy contains only "n", "w", "s", or "e" values.
assert(np.all(np.isin(policy, np.array(["n", "w", "s", "e"]))))

# Print student's solution and true solution for easier comparison / spotting of errors.
print("Student's policy:")
print(np.flip(policy.reshape((5, 5)), 0))
print("Solution policy:")
print(np.flip(solution_policy.reshape((5, 5)), 0))

print("Student's v:")
print(np.flip(v.reshape((5, 5)), 0))
print("Solution v:")
print(np.flip(solution_values.reshape((5, 5)), 0))

# Compare policy (only on states that have a single optimal direction).
states_to_check =  np.array([4, 9, 14, 17, 19, 20, 22, 24])
np.testing.assert_array_equal(policy[states_to_check], solution_policy[states_to_check])

# Compare state_values (also for terminal states --- they have to be zero!).
states_to_check = np.delete(np.arange(25), np.array([18, 23]))
np.testing.assert_array_almost_equal(v[states_to_check], solution_values[states_to_check], decimal=3)

Student's policy:
[['e' 'e' 'e' 'n' 'w']
 ['n' 'n' 'n' 'n' 'n']
 ['n' 'n' 'n' 'e' 'n']
 ['n' 'n' 'n' 'n' 'n']
 ['n' 'n' 'n' 'n' 'n']]
Solution policy:
[['e' 'e' 'e' 'n' 'w']
 ['n' 'n' 'n' 'n' 'n']
 ['n' 'n' 'n' 'e' 'n']
 ['n' 'n' 'n' 'n' 'n']
 ['n' 'n' 'n' 'n' 'n']]
Student's v:
[[7. 8. 9. 0. 9.]
 [6. 7. 8. 0. 8.]
 [5. 6. 7. 6. 7.]
 [4. 5. 6. 5. 6.]
 [3. 4. 5. 4. 5.]]
Solution v:
[[7. 8. 9. 0. 9.]
 [6. 7. 8. 0. 8.]
 [5. 6. 7. 6. 7.]
 [4. 5. 6. 5. 6.]
 [3. 4. 5. 4. 5.]]


## Exercise 2: Stochastic Environment (12 Marks)

In this exercise, we introduce stochasticity into the environment. Now, the agent is not always able to execute its actions as intended.

With probability 0.8, the agent moves as intended. However, with probability 0.2, it moves in a random direction.

For example, from grid square 0, if the agent tries to move north, with probability 0.8 the action will work as intended. But with probability 0.2, the agent's motor control system will move it in a random direction (including north). So, it will randomly try to move west, east, north or south with probability 0.05 each. Notice that the total probability of moving to square 5 (as intended) is 0.8 + 0.05 = 0.85.
 
Compute the optimal policy using Value Iteration.

Your value iteration method should output two one-dimensional numpy arrays with names `policy` and `v`, as in Exercise 1.

In [1]:
# Please write your code for Exercise 2 in this cell or in as many cells as you want ABOVE this cell.
# Your code should compute the values of policy and v from scratch when this cell is executed, 
# using the value iteration algorithm. We will mark your coursework by checking the values of 
# the variables policy and v in the hidden test cell further below. Do NOT delete this cell.

# YOUR CODE HERE

# YOUR CODE HERE
import numpy as np

class Gridworld_Stoch():
    def __init__(self,rows=5,coloumns=5,bombpos=[18],goldpos=[23]):
        self.num_r = rows
        self.num_c = coloumns
        self.shape = (self.num_r*self.num_c)
        self.bombpos = bombpos
        self.goldpos = goldpos
        self.num_fields = range(self.num_r * self.num_c)
        self.gold_reward = 10
        self.bomb_reward = -10
        self.terminalstates = self.bombpos + self.goldpos
        self.actionreward = -1
        self.actions = ["n", "e", "s", "w"]
        self.num_actions = len(self.actions)
        self.reward = np.zeros(shape=(self.shape))
        self.val = np.zeros(shape=(self.shape))
        self.policy = np.empty(25, dtype=np.unicode)
        self.gamma = 1
        self.theta = 1e-10
        
    def Reward_propagation(self, starting_pos, action):
        new_position = starting_pos
        reward = 0
        if starting_pos in self.terminalstates:
            return starting_pos,0
        else:
            if action == "n":
                candidate_pos = starting_pos + self.num_c
                new_position = starting_pos if starting_pos + 5 >= self.shape else candidate_pos
                reward = self.actionreward
            elif action == "e":
                candidate_pos = starting_pos + 1 
                new_position = starting_pos if (starting_pos + 1) % 5 == 0 else candidate_pos
                reward = self.actionreward
            elif action == "s":
                candidate_pos = starting_pos - self.num_c
                new_position = starting_pos if starting_pos - 5 < 0 else candidate_pos
                reward = self.actionreward
            elif action == "w":
                candiate_pos = starting_pos - 1
                new_position = starting_pos if (starting_pos) % 5 == 0 else candiate_pos
                reward = self.actionreward
            if new_position == self.goldpos[0]:
                reward += self.gold_reward 
            elif new_position == self.bombpos[0]:
                reward += self.bomb_reward 
        return new_position, reward
        
#     def ValueIteration(self):
#         while True:
#             deta = 0
#             prev = self.val.copy()
#             for state in self.num_fields:
#                 new_reward=[0,0,0,0]
#                 for i,act in enumerate(self.actions):
#                     if act == "n":             
#                         new_position,possible_reward = self.Reward_propagation(state,"e")
#                         new_reward[i] += 0.05*(possible_reward+self.gamma*self.val[new_position])
#                         new_position,possible_reward = self.Reward_propagation(state,"s")
#                         new_reward[i] += 0.05*(possible_reward+self.gamma*self.val[new_position])
#                         new_position,possible_reward = self.Reward_propagation(state,3)
#                         new_reward[i] += 0.05*(possible_reward+self.gamma*self.val[new_position])
#                         new_position,possible_reward = self.Reward_propagation(state,"act")
#                         new_reward[i] += 0.85*(possible_reward+self.gamma*self.val[new_position])
#                     if act == "e":             
#                         new_position,possible_reward = self.Reward_propagation(state,act)
#                         new_reward[i] += 0.85*(possible_reward+self.gamma*self.val[new_position])
#                         new_position,possible_reward = self.Reward_propagation(state,"s")
#                         new_reward[i] += 0.05*(possible_reward+self.gamma*self.val[new_position])
#                         new_position,possible_reward = self.Reward_propagation(state,3)
#                         new_reward[i] += 0.05*(possible_reward+self.gamma*self.val[new_position])
#                         new_position,possible_reward = self.Reward_propagation(state,"n")
#                         new_reward[i] += 0.05*(possible_reward+self.gamma*self.val[new_position])
#                     if act == "s":             
#                         new_position,possible_reward = self.Reward_propagation(state,"n")
#                         new_reward[i] += 0.05*(possible_reward+self.gamma*self.val[new_position])
#                         new_position,possible_reward = self.Reward_propagation(state,"e")
#                         new_reward[i] += 0.05*(possible_reward+self.gamma*self.val[new_position])
#                         new_position,possible_reward = self.Reward_propagation(state,act)
#                         new_reward[i] += 0.85*(possible_reward+self.gamma*self.val[new_position])
#                         new_position,possible_reward = self.Reward_propagation(state,3)
#                         new_reward[i] += 0.05*(possible_reward+self.gamma*self.val[new_position])
#                     if act == "w":
#                         new_position,possible_reward = self.Reward_propagation(state,"n")
#                         new_reward[i] += 0.05*(possible_reward+self.gamma*self.val[new_position])
#                         new_position,possible_reward = self.Reward_propagation(state,"e")
#                         new_reward[i] += 0.05*(possible_reward+self.gamma*self.val[new_position])
#                         new_position,possible_reward = self.Reward_propagation(state,"s")
#                         new_reward[i] += 0.05*(possible_reward+self.gamma*self.val[new_position])
#                         new_position,possible_reward = self.Reward_propagation(state,act)
#                         new_reward[i] += 0.85*(possible_reward+self.gamma*self.val[new_position])
#                 self.val[state] = max(new_reward)
#             delta = np.fabs(self.val-prev).max()
#            # print(delta)
#             if delta < self.theta:
#                 break
#         return self.val

        
    def ValueIteration(self):
        while True:
            delta = 0
            prev = self.val.copy()
            for state in self.num_fields:
                new_reward = np.zeros(4)
#check if position n,e,s,w with probabilities of stochasic environment and add this to value table during propagation
                for i,act in enumerate(self.actions):
                    for j,act in enumerate(self.actions):
                        if j!=i:
                            new_position, possible_reward = self.Reward_propagation(state,act)
                            new_reward[i] += 0.05*(possible_reward+self.gamma*self.val[new_position])
                        else:
                            new_position, possible_reward = self.Reward_propagation(state,act)
                            new_reward[i] += 0.85*(possible_reward+self.gamma*self.val[new_position])
                self.val[state] = max(new_reward)
            delta = np.fabs(self.val-prev).max()
            #print(delta)
            if delta < self.theta:
                break
        return self.val
            
                        
                
    
    
    def PolicyIteration(self):
        prev = self.val.copy()
        for state in self.num_fields:
            Policy_reward = []
            #print(Policy_reward.dtype)
            for act in self.actions:
                new_position,possible_reward = self.Reward_propagation(state,act)
                Policy_reward.append(possible_reward+self.gamma*self.val[new_position])
            Policy_reward = np.array(Policy_reward)
            Exp_reward = np.argmax(Policy_reward)
            self.policy[state] = self.actions[Exp_reward]
        return self.policy
   
    
    
    
environment = Gridworld_Stoch()

v = (environment.ValueIteration())
print(np.flip(v.reshape((5, 5)), 0))
policy = environment.PolicyIteration()
print(np.flip(policy.reshape((5, 5)), 0))

[[6.04169329 7.28756636 8.61359951 0.         8.69262311]
 [4.86185111 5.99087587 6.37082431 0.         6.46721593]
 [3.67550938 4.69621388 4.99441863 3.2189158  5.10250988]
 [2.48699534 3.40945989 3.66922967 2.64122933 3.78610115]
 [1.35979208 2.19733672 2.42878751 1.57272161 2.55202451]]
[['e' 'e' 'e' 'n' 'w']
 ['n' 'n' 'n' 'n' 'n']
 ['n' 'n' 'n' 'e' 'n']
 ['n' 'n' 'n' 'e' 'n']
 ['n' 'n' 'n' 'n' 'n']]


In [4]:
# DO NOT DELETE OR MODIFY THIS CELL!
# Your code for Exercise 2 is tested here.


## Exercise 3: Stochastic Environment with Two Pieces of Gold (8 marks)

<img src="images/bomb and two gold.png" style="width: 300px;" align="left" caption="Figure 1"/> In this exercise, we have modified the stochastic environment presented in exercise 2. A second piece of gold has been placed on grid square 12. The terminal state is reached only when **all** pieces of gold are collected or when the bomb is activated.

Compute the optimal policy for this altered environment using Value Iteration.

Hint: You will need to change your state representation in order to account for the additional piece of gold.

Your method should output two one-dimensional numpy arrays with names `policy` and `v`, as in the previous exercises. These arrays should specify the expected return and an optimal policy at the corresponding grid sqaure **before any pieces of gold are collected or a bomb is activated.** 

In [5]:
# Please write your code for Exercise 3 in this cell or in as many cells as you want ABOVE this cell.
# Your code should compute the values of policy and v from scratch when this cell is executed, 
# using the value iteration algorithm. We will mark your coursework by checking the values of 
# the variables policy and v in the hidden test cell further below. Do NOT delete this cell.

# YOUR CODE HERE

import numpy as np


class Gridworld_Stoc_gold():
    def __init__(self,rows=5,coloumns=5,bombpos=[18],goldpos=[12,23]):
        self.num_r = rows
        self.num_c = coloumns
        self.shape = (self.num_r*self.num_c)
        self.bombpos = bombpos
        self.goldpos = goldpos
        self.num_fields = range(self.num_r * self.num_c)
        self.gold_reward = 10
        self.bomb_reward = -10
        self.terminalstates = self.bombpos + self.goldpos
        self.actionreward = -1
        self.actions = ["n", "e", "s", "w"]
        self.num_actions = len(self.actions)
        self.reward = np.zeros(shape=(self.shape))
        self.val = np.zeros(shape=(self.shape))
        self.policy = np.empty(25, dtype=np.unicode)
        self.gamma = 1
        self.theta = 1e-10
        
    def Reward_propagation(self, starting_pos, action):
        new_position = starting_pos
        reward = 0
        if starting_pos in self.terminalstates:
            return starting_pos,0
        else:
            if action == "n":
                candidate_pos = starting_pos + self.num_c
                new_position = starting_pos if starting_pos + 5 >= self.shape else candidate_pos
                reward = self.actionreward
            elif action == "e":
                candidate_pos = starting_pos + 1 
                new_position = starting_pos if (starting_pos + 1) % 5 == 0 else candidate_pos
                reward = self.actionreward
            elif action == "s":
                candidate_pos = starting_pos - self.num_c
                new_position = starting_pos if starting_pos - 5 < 0 else candidate_pos
                reward = self.actionreward
            elif action == "w":
                candiate_pos = starting_pos - 1
                new_position = starting_pos if (starting_pos) % 5 == 0 else candiate_pos
                reward = self.actionreward
            if (new_position in self.goldpos): #check that new position is in the either of the gold positions
                reward += self.gold_reward 
            elif new_position == self.bombpos[0]:
                reward += self.bomb_reward 
        return new_position, reward


        

   
    def ValueIteration(self):
        while True:
            delta = 0
            prev = self.val.copy()
            for state in self.num_fields:
                
#                 check if state is in gold position 1 or 2 and call environment at this state recurvisley to continue to a terminal state
#                 track the state in value table and continue 

                if state in self.goldpos: 
                    if len(self.goldpos) > 1:
                        environment = Gridworld_Stoc_gold(self.num_r,self.num_c,self.bombpos,[self.goldpos[0] if state != self.goldpos[0] else self.goldpos[1]])
                        self.val[state] = environment.ValueIteration()[state] 
                        #print(self.val)


                    
                else:
                    new_reward = np.zeros(4)
                    for i,act in enumerate(self.actions):
                        
                        for j,act in enumerate(self.actions):
                            if j!=i:
                                new_position, possible_reward = self.Reward_propagation(state,act)
                                new_reward[i] += 0.05*(possible_reward+self.gamma*self.val[new_position])
                                
                            else:
                                new_position, possible_reward = self.Reward_propagation(state,act)
                                new_reward[i] += 0.85*(possible_reward+self.gamma*self.val[new_position])
                    self.val[state] = max(new_reward)
#                     print(self.val)
            delta = np.fabs(self.val-prev).max()
            #print(delta)
            if delta < self.theta:
                break
        return self.val


    

                        
            
       
    
    
    def PolicyIteration(self):
        prev = self.val.copy()
        for state in self.num_fields:
            Policy_reward = []
            for act in self.actions:
                new_position,possible_reward = self.Reward_propagation(state,act)
                Policy_reward.append(possible_reward+self.gamma*self.val[new_position])
            Policy_reward = np.array(Policy_reward)
            Exp_reward = np.argmax(Policy_reward)
            self.policy[state] = self.actions[Exp_reward]
        return self.policy
   
    
    
    
environment = Gridworld_Stoc_gold()

v = (environment.ValueIteration())
print(np.flip(v.reshape((5, 5)), 0))
policy = environment.PolicyIteration()
print(np.flip(policy.reshape((5, 5)), 0))

[[10.65103994 11.79603433 13.00848756  4.28547547 12.97048714]
 [11.1861353  12.32932372 12.5121464   0.         10.61568556]
 [12.28702748 13.59365638  4.99441863 12.41930416 11.19974428]
 [11.17522837 12.35165962 13.59092292 12.28122217 11.051285  ]
 [10.06409806 11.17488271 12.28045984 11.10816476  9.99389366]]
[['e' 'e' 'e' 'n' 'w']
 ['e' 's' 's' 'n' 'n']
 ['e' 'e' 'n' 'w' 'w']
 ['e' 'n' 'n' 'w' 'w']
 ['n' 'n' 'n' 'n' 'w']]


In [6]:
# DO NOT DELETE OR MODIFY THIS CELL!
# Your code for Exercise 3 is tested here.


In [7]:
print('-----------------------------------------ex1------------------------------------------')
env1 = Gridworld_Det()
%timeit                        
print('-------------------------V.I.------------------------------')
print(np.flip(env1.ValueIteration().reshape((5,5)),0))
print('-------------------------P.I.------------------------------')
print(np.flip(env1.PolicyIteration().reshape((5, 5)), 0))
print('-----------------------------------------ex2------------------------------------------')
env2 = Gridworld_Stoch()
print('-------------------------V.I.------------------------------')
print(np.flip(env2.ValueIteration().reshape((5,5)),0))
print('-------------------------P.I.------------------------------')
print(np.flip(env2.PolicyIteration().reshape((5, 5)), 0))
print('-----------------------------------------ex3------------------------------------------')
env3 = Gridworld_Stoc_gold()
print('-------------------------V.I.------------------------------')
print(np.flip(env3.ValueIteration().reshape((5,5)),0))
print('-------------------------P.I.------------------------------')
print(np.flip(env3.PolicyIteration().reshape((5, 5)), 0))
print('-----------------------------------------Completed------------------------------------------')

-----------------------------------------ex1------------------------------------------
-------------------------V.I.------------------------------
[[7. 8. 9. 0. 9.]
 [6. 7. 8. 0. 8.]
 [5. 6. 7. 6. 7.]
 [4. 5. 6. 5. 6.]
 [3. 4. 5. 4. 5.]]
-------------------------P.I.------------------------------
[['e' 'e' 'e' 'n' 'w']
 ['n' 'n' 'n' 'n' 'n']
 ['n' 'n' 'n' 'e' 'n']
 ['n' 'n' 'n' 'n' 'n']
 ['n' 'n' 'n' 'n' 'n']]
-----------------------------------------ex2------------------------------------------
-------------------------V.I.------------------------------
[[6.04169329 7.28756636 8.61359951 0.         8.69262311]
 [4.86185111 5.99087587 6.37082431 0.         6.46721593]
 [3.67550938 4.69621388 4.99441863 3.2189158  5.10250988]
 [2.48699534 3.40945989 3.66922967 2.64122933 3.78610115]
 [1.35979208 2.19733672 2.42878751 1.57272161 2.55202451]]
-------------------------P.I.------------------------------
[['e' 'e' 'e' 'n' 'w']
 ['n' 'n' 'n' 'n' 'n']
 ['n' 'n' 'n' 'e' 'n']
 ['n' 'n' 'n' 'e' '